# 1. Download SP500 Price Data

## 1.1 Import all necessary libraries

In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as pyplt
import statsmodels.api as sm
import pandas_datareader.data as web
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
from statsmodels.regression.rolling import RollingOLS

In [3]:
warnings.filterwarnings('ignore')

## 1.2 Download the SP500 Constituent Data

In [14]:
sp500 = pd.read_html("https://en.wikipedia.org/wiki/List_of_S%26P_500_companies")
print(type(sp500[0]))


<class 'pandas.core.frame.DataFrame'>


## 1.3 Clean the Data

- Isolate the ticker symbols
- Store the isoalted ticker symbols into a list

In [24]:
tickers_list = sp500[0]["Symbol"].tolist()

## 1.4 Take the start and end date to determine a range

- For this example, I will use 10 years

In [23]:
end_date = dt.datetime.today()
start_date = pd.to_datetime(end_date)-pd.DateOffset(365*10)

2024-04-26 16:57:43.939447 2014-04-29 16:57:43.939447


## 1.5 Per ticker, download the necessary information from the start date to the end date

In [26]:
df = yf.download(tickers=tickers_list, start = start_date, end = end_date)

[*********************100%%**********************]  503 of 503 completed

2 Failed downloads:
['BRK.B']: Exception('%ticker%: No timezone found, symbol may be delisted')
['BF.B']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2014-04-29 16:57:43.939447 -> 2024-04-26 16:57:43.939447)')


# 2. Calculate Different Technical Indicators per Stock

# 3. Aggregate on Monthley Level and Filter per Month the Most Liquid Stocks

# 4. Calculate Monthly Returns for Different Time-Horizons

# 5. Download Fama-French Factors; Calculate Rolling Factor Betas Per Stock

# 6.  Per Month, Create a K-means Clustering Model to group similar assets based on their Features

# 7. Per Month, Select Assets Based on the Cluster and Build a Portfolio Based on That

# 8. Visualize the Portfolio Returns and Compare That with the SP500 